# Making Transformers Efficient in Production

In previous notebooks, we've seen how transformers can be fine-tuned for various tasks. However in many situations irrespective of the metric, model is not very useful if it's too slow or too large to meet the buisness requirements of the application. The obvious alternative is to train a faster and more compact model, but with reduction comes the degradation in performance. What if we need a compact yet highly accurate model?

In this notebook we'll cover four techniques with Open Neural Network Exchange(ONNX) and ONNX Runtime(ORT) to reduce the prediction time and memory footprint of the transformers, they are:

  1. *Knowledge distillization*
  2. *Quantization*
  3. *Pruning*
  4. *Graph Optimization*

We'll also see how the techniques can be combined to produce significant performance gains. [How Roblox engineering team scaled Bert to serve 1+ Billion Daily Requests on CPUs](https://medium.com/@quocnle/how-we-scaled-bert-to-serve-1-billion-daily-requests-on-cpus-d99be090db26) and found that knowledge distillization and quantization improved the latency and throughput of their BERT classifier over a factor of 30!

To illustrate the benefits and tradeoffs associated with each technique, we'll use intent detection(important component of text-based assistants), where low latency is critical for maintaining a conversation in real time.
Along the way, we'll also learn how to create custom trainers and perform hyperparamter search, and gain a sense of what it takes to implement cutting-edge research with Transformers(lib).

## Intent Detection as a Case Study

Let's suppose we're trying to build a text-based assistant for a company's call center to deal with banking and bookings without human interaction. Base on a wide variety of natural language text(input from user), our assistant needs to classify the input into a set of predefiend actions or *intents*.

For example:

*Message*: Hey, i'd like to rent a vehichle on Nov1st.

The assitant will classify this as *car rental intent* which then triggers an action and an response.

The assistance must also have the ability to identify out of scope(oos) intents to be robust in a production environment.

As a baseline, a BERT-model has been fine-tuned with accuracy around 94% on [CLINIC150 dataset](https://arxiv.org/abs/1909.02027). This dataset includes 22,500 inscope queries across 150 intents and 10 domains like banking and travel, and also includes 1,200 *oos* intent class.

> **Note:** In practive, we would gather in-house dataset, but using public data is a greate way to iterate quickly and generate preliminary results.

First let's load the model and wrap it around a text-classification pipeline.

In [4]:
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.2/156.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from transformers import pipeline

bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline(
    task="text-classification",
    model=bert_ckpt
)

Downloading:   0%|          | 0.00/7.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
query = """Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in
Paris and I need a 15 passenger van"""
pipe(query)

[{'label': 'car_rental', 'score': 0.5490034222602844}]

The intent is good, as usual let's create a benchmark to evaluate the performance for our baseline model.

## Creating a Performance Benchmark

Like other machine learning models, deploying transformers in production environments involves trade-off among several constraints, the most common being:

*Model Performance*

How well does our model perform on a well-crafted test set that reflects production data? This is especially important when the cost of making erros is large(and best mitigated with a human in the loop) or running inference on millions of examples and small improvements to the model metrics can translate into large gains in aggregate.

*Latency*

How fast can our model deliver predictions? We usually care about latency in a real-time environment where we deail with a lot of traffic, like how stack overfloww needed a classifier to quickly [detect unwelcome comments on the website](https://stackoverflow.blog/2020/04/09/the-unfriendly-robot-automatically-flagging-unwelcoming-comments/)

*Memory*

How can we deploy billion-parameter models like GPT or T5 which requires gigabytes of disk storage and RAM? Memory plays an important role in mobile and edge devices, where we've to generate predictions without a cloud server.

Failing to address these constrains might result in:

* Poor user experience
* Balooned cloud costs for just a few user requests

To explore how the 4 different compression techniques can be used to optimizer these. Let's create a benchmark class which measures each of these quantities for a given pipeline and a test set.

In [7]:
# Skeleton of the benchmark
class PerformanceBenchmark:
  def __init__(
      self,
      pipeline,
      dataset,
      optim_type="BERT baseline"
  ):
    self.pipeline = pipeline
    self.dataset = dataset
    self.optim_type = optim_type

  def compute_accuracy(self):
    # Performance
    pass

  def compute_size(self):
    # Memory
    pass

  def time_pipeline(self):
    # Latency
    pass

  def run_benchmark(self):
    metrics = {}
    metrics[self.optim_type] = self.compute_size()
    metrics[self.optim_type].update(self.time_pipeline())
    metrics[self.optim_type].update(self.compute_accuracy())
    return metrics

We've define `optim_type` to store results of different optimization techniques. `run_benchmark()` function to collect the metrics in a dictionary.

Let's add some flesh to the skeleton by evaluating accuracy for our baseline model on the CLINC150 dataset. Let's download it.

In [8]:
dataset_path = "clinc_oos"
from datasets import get_dataset_config_names
get_dataset_config_names(
    path=dataset_path
)

Downloading:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

['small', 'imbalanced', 'plus']

In [9]:
# We'll  use the plus subset
from datasets import load_dataset
clinc = load_dataset(
    path=dataset_path,
    name="plus",
)

Downloading:   0%|          | 0.00/291k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset clinc_oos downloaded and prepared to /root/.cache/huggingface/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
clinc

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 15250
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['text', 'intent'],
        num_rows: 5500
    })
})

In [11]:
clinc["test"][2]

{'text': 'how would they say butter in zambia', 'intent': 61}

`plus` subset contains *oos* intents. Each example in the dataset has text and intent.

In [12]:
clinc["test"].features['intent']

ClassLabel(num_classes=151, names=['restaurant_reviews', 'nutrition_info', 'account_blocked', 'oil_change_how', 'time', 'weather', 'redeem_rewards', 'interest_rate', 'gas_type', 'accept_reservations', 'smart_home', 'user_name', 'report_lost_card', 'repeat', 'whisper_mode', 'what_are_your_hobbies', 'order', 'jump_start', 'schedule_meeting', 'meeting_schedule', 'freeze_account', 'what_song', 'meaning_of_life', 'restaurant_reservation', 'traffic', 'make_call', 'text', 'bill_balance', 'improve_credit_score', 'change_language', 'no', 'measurement_conversion', 'timer', 'flip_coin', 'do_you_have_pets', 'balance', 'tell_joke', 'last_maintenance', 'exchange_rate', 'uber', 'car_rental', 'credit_limit', 'oos', 'shopping_list', 'expiration_date', 'routing', 'meal_suggestion', 'tire_change', 'todo_list', 'card_declined', 'rewards_balance', 'change_accent', 'vaccines', 'reminder_update', 'food_last', 'change_ai_name', 'bill_due', 'who_do_you_work_for', 'share_location', 'international_visa', 'calend

In [13]:
intents = clinc["test"].features["intent"]

In [14]:
int2str = {i:s for i, s in enumerate(intents.names)}
int2str

{0: 'restaurant_reviews',
 1: 'nutrition_info',
 2: 'account_blocked',
 3: 'oil_change_how',
 4: 'time',
 5: 'weather',
 6: 'redeem_rewards',
 7: 'interest_rate',
 8: 'gas_type',
 9: 'accept_reservations',
 10: 'smart_home',
 11: 'user_name',
 12: 'report_lost_card',
 13: 'repeat',
 14: 'whisper_mode',
 15: 'what_are_your_hobbies',
 16: 'order',
 17: 'jump_start',
 18: 'schedule_meeting',
 19: 'meeting_schedule',
 20: 'freeze_account',
 21: 'what_song',
 22: 'meaning_of_life',
 23: 'restaurant_reservation',
 24: 'traffic',
 25: 'make_call',
 26: 'text',
 27: 'bill_balance',
 28: 'improve_credit_score',
 29: 'change_language',
 30: 'no',
 31: 'measurement_conversion',
 32: 'timer',
 33: 'flip_coin',
 34: 'do_you_have_pets',
 35: 'balance',
 36: 'tell_joke',
 37: 'last_maintenance',
 38: 'exchange_rate',
 39: 'uber',
 40: 'car_rental',
 41: 'credit_limit',
 42: 'oos',
 43: 'shopping_list',
 44: 'expiration_date',
 45: 'routing',
 46: 'meal_suggestion',
 47: 'tire_change',
 48: 'todo_list

In [15]:
int2str.get(61)

'translate'

We can use the custom int2str dict above or inbuilt int2str to translate intent ids to labels.

In [16]:
sample = clinc["test"][42]
sample

{'text': 'transfer $100 from my checking to saving account', 'intent': 133}

In [17]:
intents.int2str(sample["intent"]), int2str.get(sample["intent"])

('transfer', 'transfer')

### Compute_accuracy

In [18]:
from datasets import load_metric
accuracy_score = load_metric("accuracy")

def compute_accuracy(self):
    """This overrides the PerformanceBenchmark.compute_accuracy() method"""
    preds, labels = [], []
    for example in self.dataset:
        pred = self.pipeline(example["text"])[0]["label"]
        label = example["intent"]
        preds.append(intents.str2int(pred))
        labels.append(label)
    accuracy = accuracy_score.compute(predictions=preds, references=labels)
    print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
    return accuracy

PerformanceBenchmark.compute_accuracy = compute_accuracy

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

### Compute_size

Nex let's compute the model's size using `torch.save()`. Under the hoos PyTorch uses Python's Pickle to save the model. Pickle can be used to store any python object. Recommended way of saving a model in PyTorch is to `save_dict()`, this is a key value pair dictionray which maps weights and biases of each layer of the model. Let'e checkout the contents of state_dict().

In [19]:
list(pipe.model.state_dict().items())[:1]

[('bert.embeddings.position_ids',
  tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
            14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
            28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
            42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
            56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
            70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
            84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
            98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
           112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
           126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
           140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
           154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,

After saving the model, we can use pathlib's Path.stat() to find size of model in bytes. Let's put this all together in compute_size().

In [20]:
import torch
from pathlib import Path

def compute_size(self):
  state_dict = self.pipeline.model.state_dict()
  tmp_path = Path("model.pt")
  torch.save(state_dict, tmp_path)
  # Calculate size in MB
  size_mb = Path(tmp_path).stat().st_size / (1024 * 1024)
  # Delete temporary
  tmp_path.unlink()
  print(f"Model's size: {size_mb}")
  return {"size_mb":size_mb}

PerformanceBenchmark.compute_size = compute_size

### time_pipeline

Lastly, let's calculate the latency of the pipeline. The latency here is the time taken to pass the input to pipeline and get the predicted intent. Even though the pipeline tokenizes text, it's thousand times lesser than perdiction and negligibile.

We'll use `time` modules `perf_counter()` function from Python's `time` module. This function has a better time resolution than the `time.time()` module and well suited to get precise results.

Let's time our pipeline.

In [21]:
from time import perf_counter

for _ in range(3):
  start_time = perf_counter()
  _ = pipe(query)
  latency = perf_counter() - start_time
  print(f"Latency (ms) - {1000 * latency:.3f}")

Latency (ms) - 247.041
Latency (ms) - 209.969
Latency (ms) - 220.903


We get a varying spread of latencies every time the code runs, let's run this multiple number of times and calculate the mean and distribution to get the spread in values. We'll include a warm up phase before a timed run with CPU:

In [22]:
import numpy as np
def time_pipeline(self, query="What is the pin for my account?"):
  latencies = []
  # Warmup
  for _ in range(10):
    _ = self.pipeline(query)
  # Timed run =
  for _ in range(100):
    start_time = perf_counter()
    _ = self.pipeline(query)
    latency = perf_counter() - start_time
    latencies.append(latency)
  # Compute average staistics
  time_avg_ms = 1000 * np.mean(latencies)
  time_std_ms = 1000 * np.std(latencies)
  print(f"Average Latency (ms) - {time_avg_ms:.2f} +/- {time_std_ms:.2f}")
  return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}

PerformanceBenchmark.time_pipeline = time_pipeline

In [23]:
pb = PerformanceBenchmark(pipe, clinc['test'])
perf_metrics = pb.run_benchmark()

Model's size: 418.1499834060669
Average Latency (ms) - 88.20 +/- 11.84
Accuracy on test set - 0.867


In [24]:
perf_metrics

{'BERT baseline': {'size_mb': 418.1499834060669,
  'time_avg_ms': 88.20405813999571,
  'time_std_ms': 11.840312180573275,
  'accuracy': 0.8672727272727273}}

> **Note:** With regards to latency in this notebook, we'll compare the latency difference between models. Then we can compare the different backends(cpu vs gpu, memory etc.)

Now that we've a benchmark, next let's explore Knowledge Distillation.

## Making Models Smaller Via Knowledge Distillation

Knowledge Distilliation is a general-purpose technique to train a smaller *student* to mimic the larger *teacher* model. This was introduced in 2006 for ensemble models then popularized in 2015 for deep learning and applied to image classification and speech recognition.

With the increase in parameters with pretrained models(trillions and more), Knowledge distillation is a compression technique to compress these models and build practical applications.

### Knowledge Distillation for Fine-Tuning

#### Intution
How is knowledge distilled or transferred from teacher to student. In Fine-tuning the main idea is to augment(extend) the logits to soft probabalities. With soft probabalities we'll get probabality distribution(ex higher probablity for two intents) or information that is not accessible from labels alone. By training the student to mimic these probabalities we distill the information to the student.

#### Mathametical perspective

* We feed an input sequence X to the teacher to generate a vector of logits z(x) = [z_1(x), ..., z_N(x)]. We can convert these logits to probabalities with softmax function:

$\text{softmax}(z(x)) = \frac{\exp(z_i(x))}{\sum_{j=1}^N \exp(z_j(x))}$

* But with this softmax, we'll mostly get a single highest probability for an intent and others close to zero. With this the student can'e learn anything from truth labels.
* To soften the probabalities we'll use the hyperparameter T before applying the softmax

${p_i(x)} = \frac{\exp\left(\frac{z_i(x)}{T}\right)}{\sum_{j=1}^N \exp\left(\frac{z_j(x)}{T}\right)}$

when T=1 we recover the original softmax distribution.

*hard-vs-soft softmax distribution*

![alt](https://github.com/JpChii/nlp-with-hugging-face/blob/main/8-Making-transformers-efficient-in-production/hard-vs-soft-softmax.png?raw=1)

What's the difference between teacher and student softened probabalies?

* We can calculate this with [Kullback Leibler (KL)](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) divergence to measure the differnce between two probabalities.

$D_{\text{KL}}(p, q) = \sum_i p_i(x) \log \left(\frac{p_i(x)}{q_i(x)}\right)$

In [33]:
# Sample KL divergence
p_x = 9
q_x = 7
d_kl = p_x * (np.log(p_x / q_x))
d_kl

2.2618298545281554

In [34]:
# Sample KL divergence
p_x = 9
q_x = 3
d_kl = p_x * (np.log(p_x / q_x))
d_kl

9.887510598012987

The diergence increase when the differnce in loss is large.

* We can calculate how much is lost when we approximate the probabality distribution of studfen with the teacher. This is knowledge Distillation loss. Here we include $T^2$ because before softmax we divide the logits by T in numerator and denominator of softmax. Product of $T^2$ and KL divergence gives the loss.

$L_{\text{KD}} = T^2 \cdot D_{\text{KL}}$

* $T^2$ is the normalization factor to account for the magnitude of the gradients produced by soft label scales as 1/$T^2$.
* For classification tasks, the student loss is then a weighted average of the distillation loss with the usual cross-entropy loss $L_{\text{CE}}$ of the ground truth labels. We use a hyperparameter $\alpha$ to control control the relative strength of the losses.

$L_{\text{student}} = \alpha L_{\text{CE}} + (1 - \alpha) L_{\text{KD}}$

*Knowledge distillation process*


![alt](https://github.com/JpChii/nlp-with-hugging-face/blob/main/8-Making-transformers-efficient-in-production/entire-distillation-process.png?raw=1)

### Knowledge Distillation for Pretraining

Knowledge distillation can also be used during pretraining to create a general-purpose student that can be subsequently fine-tuned on downstream tasks. In this case Masked Language Modelling Knowledge of BERT is transferred to the student. For example, in [DistilBERT paper](https://arxiv.org/abs/1910.01108) mlm loss is augmented with a term from knowledge distillation and a cosine embedding loss to align the directions of the hidden state vectors between the teacher and student.

$L_{\text{cos}} = 1 - \cos(h_s, h_t)$

$L_{\text{DistilBERT}} = \alpha L_{\text{mlm}} + \beta L_{\text{KD}} + \gamma L_{\text{cos}}$

We already have a fine-tuned BERT model, let's use knowledge distillation to fine-tune a smaller and faster model. For this we;ve to augment the cross entropy loss with $L_{\text{KD}}$. We can do this by creating our own trainer.

### Creating a Knowledge Distillation Trainerß